In [ ]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [1]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=200)

In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

# backtest_settings = BacktestSettings(
#     array_size=1000,
#     gains_pct_filter=0,
#     total_trade_filter=40,
#     upside_filter=0,
# )

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [4]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

AttributeError: 'StaticOrderSettings' object has no attribute 'long_or_short'

In [ ]:
strat_records_df.sort_values(by='to_the_upside', ascending=False)

In [ ]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=4,
    dos_index=194,
    plot_results=True,
)

In [ ]:
# res = requests.post("http://127.0.0.1:7779/plot", json=data)

# returned_data = res.json()

# print(returned_data)